In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# !pip install --upgrade segmentation_models_pytorch

In [2]:
import os
import shutil
import pathlib
import gc
import pandas as pd
import ttach as tta
from pathlib import Path
from PIL import Image
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset
from torch.optim.swa_utils import AveragedModel, SWALR

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import timm

In [3]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
seed_everything(42)

In [5]:
ensemple_type = "No" #"No" #"mine"

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [38]:
import torchvision.transforms as T
from typing import Tuple


def resize_with_pad(image: np.array, new_shape: Tuple[int, int], padding_color: Tuple[int] = (255, 255, 255)) -> np.array:
    original_shape = (image.shape[1], image.shape[0])
    ratio = float(max(new_shape))/max(original_shape)
    new_size = tuple([int(x*ratio) for x in original_shape])

    if new_size[0] > new_shape[0] or new_size[1] > new_shape[1]:
        ratio = float(min(new_shape)) / min(original_shape)
        new_size = tuple([int(x * ratio) for x in original_shape])

    image = cv2.resize(image, new_size)
    delta_w = new_shape[0] - new_size[0]
    delta_h = new_shape[1] - new_size[1]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT,None,value=padding_color)
    return image

import torchvision.transforms as transforms


class ContrailDataset:
    def __init__(self, df, transform=None, normalize=False):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.normalize=normalize
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        
        
        
        # label_cls = 1 if label.sum() > 0 else 0
        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
#             image = resize_with_pad(image, new_shape=[528, 528])


            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))    
            
            
#         return torch.tensor(image), torch.tensor(label)
    
        if self.normalize:
            image = self.normalize_image(torch.tensor(image))
            return image, torch.tensor(label)
        else:
            return torch.tensor(image), torch.tensor(label)
    
    

In [39]:
###TImmunetplusplus model Nirjhar

n_blocks = 4

class TimmSegModel(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(
            decoder_channels[n_blocks-1],
            cfg["num_classes"], 
            kernel_size=(3, 3),
            stride=(1, 1), 
            padding=(1, 1)
        )

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

def load_model1(cfg):
    model = TimmSegModel(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model


#### Model 2 Nirjhar

n_blocks =4
class TimmSegModel2(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel2, self).__init__()

        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

def load_model2(cfg):
    model = TimmSegModel2(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model


##################

n_blocks = 4

class TimmSegModel3(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel3, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], cfg['num_classes'], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
    
def load_model3(cfg):
    model = TimmSegModel3(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model



class TimmSegModel4(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel4, self).__init__()

        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 512, 512))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
def load_model4(cfg):
    model = TimmSegModel4(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model




n_blocks_1 = 5
class TimmSegModel5(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel5, self).__init__()

        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 512, 512))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks_1+1],
                decoder_channels=decoder_channels[:n_blocks_1],
                n_blocks=n_blocks_1,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks_1-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks_1]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
def load_model5(cfg):
    model = TimmSegModel5(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model




## Rohit Model 
class Net_R(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        if cfg['model_name'] == 'Unet':
            self.model = smp.Unet(
                encoder_name=cfg["backbone"],     
                encoder_weights=None,   
                in_channels=3,  
                classes=cfg["num_classes"],
                activation=None
            )
            
        elif cfg['model_name'] == 'FPN':
            self.model = smp.FPN(
                encoder_name=cfg["backbone"],     
                encoder_weights=None,   
                in_channels=3,  
                classes=cfg["num_classes"],
                activation=None
            )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask
    
    
def load_modelr1(cfg):
    model = Net_R(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model


def load_modelr1(cfg):
    model = Net_R(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model


# def load_modelr3(cfg):
# #     weight = f"{cfg.architecture.pretrained_weights}"
#     model = Net_R(cfg)
#     model = torch.nn.DataParallel(model).cuda()
    
#     d =  torch.load(cfg['model_pth'], map_location="cpu")
#     if "model" in d:
#         model_weights = d["model"]
#     else:
#         model_weights = d
#     try:
#         model.load_state_dict(model_weights, strict=True)
#     except Exception as e:
#         print("removing unused pretrained layers")
#         for layer_name in re.findall("size mismatch for (.*?):", str(e)):
#             model_weights.pop(layer_name, None)
#         model.load_state_dict(model_weights, strict=False)

#     print(f"Weights loaded from: {cfg.architecture.pretrained_weights}")



#     if idx in [11, 12]:
#         model = TimmSegModelR1(cfg)
#         model = torch.nn.DataParallel(model).cuda()
#         model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
#     else:
#         model = Net(cfg)    
#         model = torch.nn.DataParallel(model).cuda()
#         model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])

class TimmSegModelR1(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModelR1, self).__init__()

        self.n_blocks = 4
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:self.n_blocks+1],
                decoder_channels=decoder_channels[:self.n_blocks],
                n_blocks=self.n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[self.n_blocks-1], 
                      cfg['num_classes'],
                      kernel_size=(3, 3),
                      stride=(1, 1),
                      padding=(1, 1)
            ),
            nn.UpsamplingBilinear2d(scale_factor=1)
        )

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:self.n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features


    
def load_modelr2(cfg):
    model = TimmSegModelR1(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model

### Ioannis

In [40]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice


def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform

def get_transform2(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_LINEAR),
    ], p=1.0)
    return transform




In [41]:
val_df = pd.read_csv("../../input/data_utils/val_df_filled.csv")
val_df_full = val_df.copy()

# val_df = val_df.loc[val_df['class'] == 1].reset_index(drop=True)
val_dups = np.load("../../input/data_utils/dups_val.npy")

val_dups = [int(val_id) for val_id in val_dups]

In [42]:
# val_df = val_df.loc[~val_df['id'].isin(val_dups)].reset_index(drop=True)
print(val_df.shape, val_df_full.shape)

(1856, 4) (1856, 4)


In [43]:
# val_df.to_csv(f"../../output/final_preds/val_df_pos.csv", index=False)

In [44]:
CFGS1 = [
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01",
#         'model_func': load_modelr1,
#         'tta': False,
#         'normalize': False
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s2/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_02_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_03_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s2_swa/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_04_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s8_swa/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_05_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s9_swa/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_06_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
    
    
#     {
#         'model_name': 'FPN',
#         'backbone': 'efficientnet-b6',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_swa/FPN/efficientnet-b6-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_07_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, # nir_07_tta hvflip
    
 
#     {
#         'model_name': 'FPN',
#         'backbone': 'efficientnet-b6',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_swa_s2/FPN/efficientnet-b6-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_07_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, # nir_07_tta
    

    
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01_tta",
#         'model_func': load_model4,
#         'tta': True,
#         'normalize': False
#     },
    
#       {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01",
#         'model_func': load_model4,
#         'tta': False,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01_tta",
#         'model_func': load_model4,
#         'tta': False,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02_tta",
#         'model_func': load_model2,
#         'tta': True,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_03_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04_tta", 
#         'model_func': load_model1,
#         'tta': True,
#         'normalize': False
#     }, 
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/fullfinetune-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2cv670-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_05_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ema_ema-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_06_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/pseudostage2-effb8modelv2fold0_tf_efficientnet_b8_best_ctrl_epoch-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/pseudostage2-effb8modelv2fold0_tf_efficientnet_b8_best_ctrl_epoch-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07", 
#         'model_func': load_model3,
#         'tta': False,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01",
#         'model_func': load_model4,
#         'tta': False,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02",
#         'model_func': load_model2,
#         'tta': False,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_03", 
#         'model_func': load_model3,
#         'tta': False,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04", 
#         'model_func': load_model1,
#         'tta': False,
#         'normalize': False
#     }, 
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/fullfinetune-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2cv670-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_05", 
#         'model_func': load_model3,
#         'tta': False,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/swa-pseudostage3-effb8modelv2fold0v2_tf_efficientnet_b8_best_ctrl_epoch-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/fullfinetune-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2cv670-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_08_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
    
    

#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output2/exp_10_swa/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_x1",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output2/exp_10_s1_v1_pseudo/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_s1",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output2/exp_10_s1_v1_pseudo_swa/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_s2",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output2/exp_10_swa_2_v1/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_s3",
#         'model_func': load_modelr1, # flip
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output2/exp_10_swa_2_v2/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_s4",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },

    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output2/exp_10_s1_v1_pseudo/Unet/efficientnet-b7-256/checkpoint_dice_ensemble_best0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_s1_1",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output2/exp_10_s1_v1_pseudo_swa/Unet/efficientnet-b7-256/checkpoint_dice_ensemble_best0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_s2_2",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output2/exp_10_swa_2_v1/Unet/efficientnet-b7-256/checkpoint_dice_ensemble_best0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_s3_3",
#         'model_func': load_modelr1, # flip
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output2/exp_10_swa_2_v2/Unet/efficientnet-b7-256/checkpoint_dice_ensemble_best0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_s4_4",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },
    
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output2/exp_10_swa_2_v2/Unet/efficientnet-b7-256/checkpoint_dice_ensemble_best0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_s4_4",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output3/exp_10_s1_full/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_10_tta",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },
    
#      {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output3/exp_10_s1_pos/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_11_tta",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/IRPrtrain-noPLs3_maxvit_small_tf_512_best_ctrl_epoch-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_09_tta",
#         'model_func': load_model4,
#         'tta': True,
#         'normalize': False
#     },
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output3/exp_12/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_12_tta",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },
    
]

In [45]:
CFGS1 = [
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/fullfinetune-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2cv670-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_05_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
    
    
    {
        'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b8',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ema_ema-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_06_tta", 
        'model_func': load_model3,
        'tta': True,
        'normalize': False
    },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/IRPrtrain-noPLs3_maxvit_small_tf_512_best_ctrl_epoch-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_09_tta",
#         'model_func': load_model4,
#         'tta': True,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_l2_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nopretrain-effl2-512-unetplus-fold0-s3_tf_efficientnet_l2_ns_best_ctrl_epoch-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_10_tta", 
#         'model_func': load_model5,
#         'tta': True,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s2/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_02_tta_f",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output3/exp_12_3/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_12_2_tta",
#         'model_func': load_modelr1, # d4
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/IRpretrainv2-pseudotrain_v2-s4_tf_efficientnet_b8_best_ctrl_epoch-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_11_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
    
]

In [46]:
final_preds1 = []

for idx, cfg in enumerate(CFGS1):   

#     if idx <= 10:
#         continue
        
    print(cfg)
    val_transform = get_transform(cfg['img_size'])
    val_transform2 = get_transform2(cfg['img_size'])

    valid_dataset = ContrailDataset(val_df, transform=val_transform, normalize=False)  
    if cfg['normalize'] and ("ioa_" in cfg['call_sign']):
        valid_dataset = ContrailDataset(val_df, transform=val_transform2, normalize=True)  
#         valid_dataset = ContrailDataset(val_df, transform=val_transform, normalize=True)  


    
    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )
    
    
    model = cfg['model_func'](cfg)        
      
    if cfg['tta']:
        if "roh_" in cfg['call_sign']:

#             model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
            model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
#             model3 = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')

        else:
            model = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
#             model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
#             model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')


    
#     model1.to(device)
#     model2.to(device)
#     model3.to(device)
#     model1.eval()
#     model2.eval()
#     model3.eval()
    
    
    model.to(device)
    model.eval()
    
    
    preds = []
    masks_ = []      
        
    for index, (images, masks) in enumerate(tqdm(valid_loader)):  
        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        if cfg['img_size'][0] != 256:
            masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 

        masks_.append(torch.squeeze(masks, dim=1))
        with torch.inference_mode():
#             images = torchvision.transforms.CenterCrop(size)
            
#             images = torchvision.transforms.functional.center_crop(images, output_size= [512,512])
    
            images = torch.nn.functional.interpolate(images, size=cfg['img_size'][0], mode='nearest')
            pred = model(images).sigmoid() 
#             pred2 = model2(images).sigmoid() 
#             pred3 = model3(images).sigmoid() 
            
#             pred = (pred1 + pred2 + pred3) / 3
                
            pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest')
            preds.append(torch.squeeze(pred, dim=1))
        
            
            
    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
        
    print(model_preds.shape)

#     np.save(f"../../output/final_preds/{cfg['call_sign']}", model_preds.detach().cpu().numpy())
#     if idx == 0:
#         np.save(f"../../output/final_preds/val_masks", model_masks.detach().cpu().numpy())

    
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, model_preds, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    
        
    print(best_dice_score, best_threshold)
    final_preds1.append(model_preds)
    
#     del model1, model2, model3
    del model 
    torch.cuda.empty_cache()
    gc.collect()

{'model_name': 'Unet++', 'backbone': 'tf_efficientnet_b8', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ema_ema-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin', 'threshold': 0.79, 'call_sign': 'nir_06_tta', 'model_func': <function load_model3 at 0x7f6a54a23be0>, 'tta': True, 'normalize': False}


  0%|          | 0/58 [00:00<?, ?it/s]

torch.Size([1856, 256, 256])
0.67619073 0.91


In [52]:
### 0.67995                                                                                                           

In [15]:
def get_small_area(wt, wt_all, mask):
    best_threshold = 0.0
    best_dice_score = 0.0
    best_area = 0.0
    for i in range(500):
#         wt1 = wt
#         for idx, val in enumerate(wt1):
#             if val.sum() <= i:
#                 wt1[idx, :, :] = 0
                
        wt1 = wt
        for idx, val in enumerate(wt_all):
            if val.sum() <= i:
                wt1[idx, :, :] = 0
                
        wt1 = wt1.cuda()
        for threshold in [i / 100 for i in range(101)] :
            score = dice_coef(model_masks, wt1, thr=threshold).cpu().detach().numpy() 
            if score > best_dice_score:
                best_dice_score = score
                best_threshold = threshold
                best_area = i
        print(best_dice_score, best_threshold, best_area)
                
    print(best_dice_score, best_threshold, best_area)    
    
    
    

In [16]:
# t_3 = [
#     "roh_02_tta_3",
#     "roh_11_tta_3",
#     "nir_05_tta_3",
#     "nir_06_tta_3",
#     "nir_09_tta_3",
# ]

# t_5 = [
#     "roh_02_tta_5",
#     "roh_11_tta_5",
#     "nir_05_tta_5",
#     "nir_06_tta_5",
#     "nir_09_tta_5",
# ]



# t3_preds = []
# for idx, sign in tqdm(enumerate(t_3), total=len(t_3)):
#     sign = f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.pt"
#     t3_preds.append(torch.load(sign).cuda())
    
    
    
# t5_preds = []
# for idx, sign in tqdm(enumerate(t_5), total=len(t_5)):
#     sign = f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.pt"
#     t5_preds.append(torch.load(sign).cuda())

In [17]:
# t3_preds = torch.stack(t3_preds).mean(0).squeeze(dim=1)
# t5_preds = torch.stack(t5_preds,dim=0).mean(0).squeeze(dim=1)

In [18]:
# t5_preds.shape

In [19]:
def boost_dice(params, TH=None):    
    
    for index, val in enumerate(params.keys()):
        if index == 0:            
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
    
#     for i, val in enumerate(preds):
#         if val.sum() <= 28:
#             preds[i, :, :] = 0
    
#     print(preds.shape, model_masks.shape)


#     wt1 = (preds)
#     get_small_area(preds, preds, model_masks)
#     print(preds.shape)    
    
    
    if TH:
        best_threshold = TH
        preds1 = (preds).double()
        best_score = dice_coef(model_masks, preds1, thr=TH).cpu().detach().numpy() # get_score(labels, preds) 
    else:
        best_threshold = 0.0
        best_score = 0.0
        for threshold in [i / 100 for i in range(101)] :              
            if threshold < 0.33 or threshold > 0.45:
                continue
#         for threshold in [0.37, 0.38, 0.39, 0.4]:              

            preds1 = ((preds)>threshold).double()            
            score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
            if score > best_score:
                best_score = score
                best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [47]:
# call_signs1 = [    
# #     "roh_10_tta",
# #     "roh_02_tta", 
    
#     "roh_11_tta_full",

    
#     "nir_05",
#     "nir_06_tta",
#     "nir_09",
#     "121_tta",
#     "162_tta",
#     "166_tta",
#     "125", 
#     "196",
# #     "127",
    
#     "204",
    
#     "Unetpp-convnext_base_preds_tta_06638",
# #     "Unet-tf_efficientnet_l2_ns_tta_06570"
#     # above this 
    
# #     "195",
# #     "198",
# #     "184", # 1
# #     "174",
# #     "150",
# #     "138"    
    
        
# #     "195_tta",
# #     "198_tta",
# #     "184_tta",
# #     "174_tta",
# #     "150_tta",
# #     "138_tta"
    
    
# ] 



# call_signs1 = [    
# #     "roh_10_tta",
#     "roh_12_tta", 
#     "roh_02_tta",
#     "roh_11_tta",
#     "nir_05_tta",
#     "nir_06_tta",
#     "nir_09_tta",
#     "121_tta",
#     "162_tta",
#     "166_tta",
#     "125_tta", 
#     "196_tta",
# #     "127_tta",
#     "204", # tta
    
#     "Unetpp-convnext_base_preds_tta_06638",
# #     "Unet-tf_efficientnet_l2_ns_tta_06570"

# ] 

# call_signs2 = [  
#     "roh_11_tta", # 6448363
# #     "nir_06_tta", # 67619073
#     "nir_09_tta", # 6722194
#     "121_tta", # 6868992
#     "162_tta", # 6853779
#     "166_tta", # 678758
#     "229_tta",  # 65417683
# #     "235_tta", # 6874996
#     "ioa_exp55_tta", # 6665687
#     "roh_12_2_tta" # 67948383
#     "nir_05_tta" # 686
# ] 


call_signs2 = [  

#     "166_tta", # 678758
#     "roh_11_tta", # 6448363
    "282_tta",
    "283_tta",
    "284_tta",
    "265_tta"
] 

call_signs1 = [
    
    "nir_06_tta",
    "nir_09_tta", # 6722194
    "121_tta", # 6868992
    "162_tta", # 6853779
    "229_tta",  # 65417683
    "235_tta",
    "272_tta",
#     "255_tta"
    "ioa_exp55_tta", # 6665687
    
#     "282_tta",
#     "283_tta",
#     "284_tta",
    "285_tta",
#     "287_tta",
#     "289_tta",
#     "275_tta"
    
    
#     "265_tta"
    
    
    
#### 0.7018820643424988
#     "166_tta"

#     "166_tta",
#     "roh_11_tta", # 6448363
#     "255_tta",
#     "convnext_base_preds_tta_06651",
#     "convnext_large_preds_tta_06711"
]


#     params = {}
#     params[f"w1"] = 0.12527945585979453
#     params[f"w2"] = 0.4274708716604963
#     params[f"w3"] = 0.60359403891758
#     params[f"w4"] = 0.598088267045536
#     params[f"w5"] = 0.8735733730972042
#     params[f"w6"] = 0.2509448081151351
#     params[f"w7"] = 0.4990068077158164
#     params[f"w8"] = 0.9475873953049087
#     params[f"w9"] = 0.5673029916578154







model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).cuda() #.flatten().cuda() 

preds = []
for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):    
    wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).cuda() 
    
#     for i, val in enumerate(wt):
#         if val.sum() <= areas[idx]:
#             wt[i, :, :] = 0
    
    
    
#     get_small_area(wt, wt1, model_masks)
    

    preds.append(wt)    
    score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
    print(score)
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, wt, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    print(best_dice_score, best_threshold)
    
    
    
    
#     print(score)
    
    
    # # preds1.append(preds2)

#     final_preds = preds
#     final_preds = torch.stack(final_preds).mean(dim=0)
#     score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 

#     print("0.5 TH Score: ", score)


#     best_threshold = 0.0
#     best_dice_score = 0.0
#     for threshold in [i / 100 for i in range(101)] :
#         if threshold < 0.35 or threshold > 0.51:
#             continue

#         score = dice_coef(model_masks, final_preds, thr=threshold).cpu().detach().numpy() 
#         if score > best_dice_score:
#             best_dice_score = score
#             best_threshold = threshold

#     print(best_dice_score, best_threshold)

  0%|          | 0/9 [00:00<?, ?it/s]

0.6752145
0.67619073 0.91
0.67202777
0.6722194 0.16
0.68600667
0.6868992 0.44
0.6818373
0.6853779 0.37
0.64944065
0.65417683 0.25
0.6838827
0.6874996 0.25
0.6833817
0.6841299 0.57
0.66382974
0.6665687 0.3
0.6769075
0.6769075 0.5


In [26]:
# best_score = 0
# best_idx = 0

# for idx, sign in tqdm(enumerate(call_signs2), total=len(call_signs2)):    
    
#     preds1 = []
#     wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).cuda() 
#     preds1.append(wt)    
#     score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
# #     print(score)
#     best_threshold = 0.0
#     best_dice_score = 0.0
#     for threshold in [i / 100 for i in range(101)] :
#         score = dice_coef(model_masks, wt, thr=threshold).cpu().detach().numpy() 
#         if score > best_dice_score:
#             best_dice_score = score
#             best_threshold = threshold
#     print(best_dice_score, best_threshold)
    
#     final_preds = preds  + preds1


#     def objective(trial):
#         TH = 0.38 #0.38 #0.38 #0.38 #0.39

#         params = {}    
#         for i in range(len(final_preds)):
#             params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)
#         score, best_threshold  = boost_dice(params, TH=TH)
#         params['threshold'] = best_threshold
#         if not TH:
#             print(params)

#         return score


#     study = optuna.create_study(direction='maximize') #0.7013314962387085
#     study.optimize(objective, n_trials=300)
#     best_params = study.best_params

#     if study.best_trial.values[0] > best_score:
#         best_score = study.best_trial.values[0]
#         best_idx = idx
#         print(idx, best_score)
    

In [117]:
best_score, best_idx # 0.701389491558075

(0.7018152475357056, 0)

In [ ]:
# [I 2023-08-09 12:45:11,105] Trial 172 finished with value: 0.7018820643424988 and parameters: {'w1': 0.9601814501763974, 'w2': 0.3756774047012578, 'w3': 0.7824389227828425, 'w4': 0.5792644776493453, 'w5': 0.7771259772426796, 'w6': 0.711103975135206, 'w7': 0.5081077074379339, 'w8': 0.5616052608027059}. Best is trial 172 with value: 0.7018820643424988.
best_score

In [30]:
final_preds = preds

In [33]:
def objective(trial):
    TH = 0.38 #0.38 #0.38 #0.38 #0.38 #0.38 #0.38 #0.39
    
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)
        
        
    
#     params = {}
#     params[f"w1"] = 0.33767097199562696
# #     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)
#     params[f"w2"] = 0.53361360969572
# #     params[f"w2"] = trial.suggest_float(f'w2', 0, 1)
#     params[f"w3"] = 0.3991080538770059
# #     params[f"w3"] = trial.suggest_float(f'w3', 0, 1)
#     params[f"w4"] = 0.6717493728460793
# #     params[f"w4"] = trial.suggest_float(f'w4', 0, 1)
#     params[f"w5"] = 0.6869441529028858
# #     params[f"w5"] = trial.suggest_float(f'w5', 0, 1)
#     params[f"w6"] = 0.9181199687590184
# #     params[f"w6"] = trial.suggest_float(f'w6', 0, 1)
#     params[f"w7"] = 0.7052727698454961
# #     params[f"w7"] = trial.suggest_float(f'w7', 0, 1)
#     params[f"w8"] = 0.48443588742431637
# #     params[f"w8"] = trial.suggest_float(f'w8', 0, 1)






#     params = {'w1': 0.18972963733219855,
#      'w2': 0.6372756138259427,
#      'w3': 0.857310296865279,
#      'w4': 0.9627606108075277,
#      'w5': 0.5146613105123311,
#      'w6': 0.5231847335562232,
#      'w7': 0.6840001283288519,
#      'w8': 0.5488904084538432,
#      'w9': 0.48779915288000425}

    
    
    score, best_threshold  = boost_dice(params, TH=TH)
    params['threshold'] = best_threshold
    if not TH:
        print(params)
        
    return score

In [34]:
best = []
for i in tqdm(range(15), total=15):

    study = optuna.create_study(direction='maximize') #0.7013314962387085
    study.optimize(objective, n_trials=350)
    best_params = study.best_params
    best_score = study.best_trial.values[0]

    best_dict = {}
    best_dict['params'] = best_params
    best_dict['score'] = best_score

    best.append(best_dict)
    print(best_dict)
    

  0%|          | 0/15 [00:00<?, ?it/s]

[I 2023-08-09 23:50:42,805] A new study created in memory with name: no-name-84bec308-d636-4663-8683-1e115c84b28d
[I 2023-08-09 23:50:42,844] Trial 0 finished with value: 0.6988447904586792 and parameters: {'w1': 0.2664448931648541, 'w2': 0.8238285683667591, 'w3': 0.9188407876794906, 'w4': 0.5898799006216091, 'w5': 0.5237476991279606, 'w6': 0.3356656114055486, 'w7': 0.10003732717688563, 'w8': 0.13772335124153845, 'w9': 0.16096433007523325}. Best is trial 0 with value: 0.6988447904586792.
[I 2023-08-09 23:50:42,882] Trial 1 finished with value: 0.699186384677887 and parameters: {'w1': 0.21377629097962536, 'w2': 0.06198482205205158, 'w3': 0.878069597831673, 'w4': 0.5774201195645147, 'w5': 0.4387173153856653, 'w6': 0.29012733062387663, 'w7': 0.7736057322210289, 'w8': 0.4537806278449227, 'w9': 0.2206423569835173}. Best is trial 1 with value: 0.699186384677887.
[I 2023-08-09 23:50:42,919] Trial 2 finished with value: 0.6959511637687683 and parameters: {'w1': 0.8609514543071052, 'w2': 0.7922

[I 2023-08-09 23:50:44,163] Trial 22 finished with value: 0.7000024914741516 and parameters: {'w1': 0.6120117108290911, 'w2': 0.5524000122857198, 'w3': 0.957176216754886, 'w4': 0.5117232449197471, 'w5': 0.8905889612921545, 'w6': 0.4560839686025614, 'w7': 0.393313722251406, 'w8': 0.8924193977087389, 'w9': 0.1124002883746596}. Best is trial 3 with value: 0.7011540532112122.
[I 2023-08-09 23:50:44,235] Trial 23 finished with value: 0.7003485560417175 and parameters: {'w1': 0.3880544791190983, 'w2': 0.5260143717011875, 'w3': 0.999905403252357, 'w4': 0.34420428079173526, 'w5': 0.6980843877306597, 'w6': 0.5855800227102972, 'w7': 0.3665653999785457, 'w8': 0.8959539166336185, 'w9': 0.27231757347180163}. Best is trial 3 with value: 0.7011540532112122.
[I 2023-08-09 23:50:44,307] Trial 24 finished with value: 0.7005725502967834 and parameters: {'w1': 0.49793056321918433, 'w2': 0.6353768197311479, 'w3': 0.8813470689682196, 'w4': 0.4051135034028854, 'w5': 0.8287076162306608, 'w6': 0.79595251287183

[I 2023-08-09 23:50:45,787] Trial 44 finished with value: 0.701087474822998 and parameters: {'w1': 0.1567799547757046, 'w2': 0.8060808750781718, 'w3': 0.6093458042591196, 'w4': 0.8706021265949563, 'w5': 0.5089504085046315, 'w6': 0.8586514528467156, 'w7': 0.5982733225488952, 'w8': 0.5906476125720764, 'w9': 0.05869341115323662}. Best is trial 41 with value: 0.7012290954589844.
[I 2023-08-09 23:50:45,863] Trial 45 finished with value: 0.7012189626693726 and parameters: {'w1': 0.12551339474005602, 'w2': 0.8126246289919904, 'w3': 0.5812345318468052, 'w4': 0.8595328747348727, 'w5': 0.6483337296822634, 'w6': 0.8544282766328782, 'w7': 0.6096845406112701, 'w8': 0.5937489599552326, 'w9': 0.05567155427496227}. Best is trial 41 with value: 0.7012290954589844.
[I 2023-08-09 23:50:45,938] Trial 46 finished with value: 0.7011944651603699 and parameters: {'w1': 0.11340878591667887, 'w2': 0.8213005450027255, 'w3': 0.5578076637299657, 'w4': 0.8639425159127799, 'w5': 0.6562963848391897, 'w6': 0.957470238

[I 2023-08-09 23:50:47,442] Trial 66 finished with value: 0.7014609575271606 and parameters: {'w1': 0.3930372521531095, 'w2': 0.6590557986162332, 'w3': 0.42431412591781786, 'w4': 0.7520201437993181, 'w5': 0.781605532163571, 'w6': 0.9238805371352642, 'w7': 0.470918537885173, 'w8': 0.31177919425815265, 'w9': 0.00045863228348022936}. Best is trial 49 with value: 0.7015460729598999.
[I 2023-08-09 23:50:47,517] Trial 67 finished with value: 0.7012545466423035 and parameters: {'w1': 0.31973716504537814, 'w2': 0.7525257112941829, 'w3': 0.31586617854289667, 'w4': 0.7943983486686199, 'w5': 0.7364577743897399, 'w6': 0.8822989411920069, 'w7': 0.5668145669590879, 'w8': 0.3995290603143184, 'w9': 0.135232568310684}. Best is trial 49 with value: 0.7015460729598999.
[I 2023-08-09 23:50:47,592] Trial 68 finished with value: 0.7012840509414673 and parameters: {'w1': 0.21513733575233607, 'w2': 0.6818034913392236, 'w3': 0.46363762466526515, 'w4': 0.8324420376808068, 'w5': 0.7142163395058171, 'w6': 0.82253

...

[I 2023-08-09 23:57:31,438] Trial 269 finished with value: 0.7006929516792297 and parameters: {'w1': 0.5213923298989893, 'w2': 0.38381490333708135, 'w3': 0.32111490249259406, 'w4': 0.042429914233613086, 'w5': 0.583002745101887, 'w6': 0.9244637137603454, 'w7': 0.6469578632953021, 'w8': 0.36631849008958434, 'w9': 0.10327288201236405}. Best is trial 233 with value: 0.7017121911048889.
[I 2023-08-09 23:57:31,522] Trial 270 finished with value: 0.7012508511543274 and parameters: {'w1': 0.4960519727260001, 'w2': 0.4833947603561808, 'w3': 0.36249580466882153, 'w4': 0.6704653712749928, 'w5': 0.5037215245290522, 'w6': 0.8263420187781053, 'w7': 0.6783148520036093, 'w8': 0.26990110660494804, 'w9': 0.3074344378358007}. Best is trial 233 with value: 0.7017121911048889.
[I 2023-08-09 23:57:31,610] Trial 271 finished with value: 0.7013488411903381 and parameters: {'w1': 0.5738237730425355, 'w2': 0.42644327153122635, 'w3': 0.42805931071151804, 'w4': 0.6110148414954756, 'w5': 0.541395143256531, 'w6': 0

[I 2023-08-09 23:57:33,315] Trial 291 finished with value: 0.7014726400375366 and parameters: {'w1': 0.41258437321692587, 'w2': 0.5615033958054805, 'w3': 0.3810755834882521, 'w4': 0.647830510677085, 'w5': 0.5930132444698774, 'w6': 0.9587478636005224, 'w7': 0.6499377240100321, 'w8': 0.38613118894464427, 'w9': 0.21196724897990227}. Best is trial 233 with value: 0.7017121911048889.
[I 2023-08-09 23:57:33,400] Trial 292 finished with value: 0.7011140584945679 and parameters: {'w1': 0.5056725941659805, 'w2': 0.5797099703324434, 'w3': 0.19167621938919752, 'w4': 0.5058422478352183, 'w5': 0.5342936967760403, 'w6': 0.9420631270215923, 'w7': 0.6805462094142517, 'w8': 0.07565082034037957, 'w9': 0.018608957892847913}. Best is trial 233 with value: 0.7017121911048889.
[I 2023-08-09 23:57:33,485] Trial 293 finished with value: 0.7008931040763855 and parameters: {'w1': 0.26156146467497776, 'w2': 0.3604411160373487, 'w3': 0.31156486890049145, 'w4': 0.6294408982550027, 'w5': 0.3084285901877113, 'w6': 0

[I 2023-08-09 23:57:35,202] Trial 313 finished with value: 0.7014581561088562 and parameters: {'w1': 0.5108856152997575, 'w2': 0.4825173848757783, 'w3': 0.09956173878986012, 'w4': 0.7914899934932434, 'w5': 0.35079420492104935, 'w6': 0.9815778338278199, 'w7': 0.6140196825155293, 'w8': 0.33888997181856406, 'w9': 0.0780751796738513}. Best is trial 233 with value: 0.7017121911048889.
[I 2023-08-09 23:57:35,288] Trial 314 finished with value: 0.700743556022644 and parameters: {'w1': 0.5316846676271879, 'w2': 0.3928962963473361, 'w3': 0.15747893954964912, 'w4': 0.6409268931871851, 'w5': 0.5672859862067454, 'w6': 0.9992571311107671, 'w7': 0.4273771868101698, 'w8': 0.0231583467758949, 'w9': 0.018116779431782193}. Best is trial 233 with value: 0.7017121911048889.
[I 2023-08-09 23:57:35,373] Trial 315 finished with value: 0.7009766101837158 and parameters: {'w1': 0.47938457377766314, 'w2': 0.45811978105618295, 'w3': 0.1896437093933366, 'w4': 0.6594974467398008, 'w5': 0.5197536539984756, 'w6': 0.

[I 2023-08-09 23:57:37,149] Trial 335 finished with value: 0.7002028226852417 and parameters: {'w1': 0.5442953602647057, 'w2': 0.393999504528843, 'w3': 0.18310344332601497, 'w4': 0.485545874966012, 'w5': 0.5347737749474165, 'w6': 0.6122641614965996, 'w7': 0.32232533838006466, 'w8': 0.11182370988924066, 'w9': 0.18734228890109528}. Best is trial 233 with value: 0.7017121911048889.
[I 2023-08-09 23:57:37,240] Trial 336 finished with value: 0.7004215121269226 and parameters: {'w1': 0.597287243475783, 'w2': 0.5682469123687194, 'w3': 0.1200038925636038, 'w4': 0.6537801732427877, 'w5': 0.6555457371956284, 'w6': 0.5395352156702624, 'w7': 0.709532599592151, 'w8': 0.26261089903530144, 'w9': 0.04027809074221706}. Best is trial 233 with value: 0.7017121911048889.
[I 2023-08-09 23:57:37,328] Trial 337 finished with value: 0.7007541656494141 and parameters: {'w1': 0.9239641709941088, 'w2': 0.43370722834913156, 'w3': 0.25743263625395696, 'w4': 0.9799227414977407, 'w5': 0.5074493556582913, 'w6': 0.909

{'params': {'w1': 0.4850705328530502, 'w2': 0.4262182826383574, 'w3': 0.16299491250734852, 'w4': 0.6226815813681976, 'w5': 0.5658022565193405, 'w6': 0.9791491651026963, 'w7': 0.6594227296700484, 'w8': 0.3530813798816731, 'w9': 0.02032414234748772}, 'score': 0.7017121911048889}


In [35]:
# 0.0.7020075917243958

best_params # 0.7014082074165344

# # {'w1': 0.2223815786205381, 'w2': 0.1460101485299246, 'w3': 0.26070970307361535, 'w4': 0.4027457337153253, 'w5': 0.6526582421453944, 'w6': 0.2822502717916394}


# # params

# {'w1': 0.4274708716604963,
#  'w2': 0.5950793403696619,
#  'w3': 0.5981031818823078,
#  'w4': 0.9052807157209483,
#  'w5': 0.24903814629277216,
#  'w6': 0.5675462825594632,
#  'w7': 0.49869913358097817,
#  'w8': 0.9473968763799567,
#  'w9': 0.02603252721027552,
#  'w10': 0.11450732040441257}

# for val in best:
#     if val['score'] > 0.7013:
#         print(val)

{'w1': 0.4850705328530502,
 'w2': 0.4262182826383574,
 'w3': 0.16299491250734852,
 'w4': 0.6226815813681976,
 'w5': 0.5658022565193405,
 'w6': 0.9791491651026963,
 'w7': 0.6594227296700484,
 'w8': 0.3530813798816731,
 'w9': 0.02032414234748772}

In [105]:
0.7020075917243958 - 0.7016780972480774

0.0003294944763183594

In [ ]:
params = {}
params[f"w1"] = 0.12527945585979453
params[f"w2"] = 0.4274708716604963
params[f"w3"] = 0.60359403891758
params[f"w4"] = 0.598088267045536
params[f"w5"] = 0.8735733730972042
params[f"w6"] = 0.2509448081151351
params[f"w7"] = 0.4990068077158164
params[f"w8"] = 0.9475873953049087
params[f"w9"] = 0.5673029916578154

In [ ]:
params

In [ ]:
params = {}    
#     for i in range(len(final_preds)):
#         params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)

params[f"w1"] = 0.17983155241560006
#     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)
params[f"w2"] = 0.3921968863625922
#     params[f"w2"] = trial.suggest_float(f'w2', 0, 1)
params[f"w3"] = 0.5467899983419907
#     params[f"w3"] = trial.suggest_float(f'w3', 0, 1)
params[f"w4"] = 0.770948318764219
#     params[f"w4"] = trial.suggest_float(f'w4', 0, 1)
params[f"w5"] = 0.9994494733078347
#     params[f"w5"] = trial.suggest_float(f'w5', 0, 1)
params[f"w6"] = 0.6290244452681286
#     params[f"w6"] = trial.suggest_float(f'w6', 0, 1)
params[f"w7"] = 0.48637848861057426
#     params[f"w7"] = trial.suggest_float(f'w7', 0, 1)
params[f"w8"] = 0.19103711312728738
#     params[f"w8"] = trial.suggest_float(f'w8', 0, 1)
params[f"w9"] = 0.6303399534920496
#     params[f"w9"] = trial.suggest_float(f'w9', 0, 1)
params

In [ ]:
# 0.37

# {'w1': 0.839014852140699, 'w2': 0.5011350487644706, 'w3': 0.5425134900557348, 'w4': 0.5452545452276221, 'w5': 0.6026139561761139, 'w6': 0.9831197346233449, 'w7': 0.8036481597462362, 'w8': 0.5819171235801198}
# 0.7003488540649414


# Trial 1 finished with value: 0.6991791129112244 and parameters: {'w1': 0.2047525215621725}. Best is trial 1 with value:
# 0.0.6996479034423828
# 0.36
# {
# 'w1': 0.36485377740463365,
# 'w2': 0.5716506827395362,
# 'w3': 0.84301760289886,
# 'w4': 0.5944703175254541,
# 'w5': 0.8096022495041699
# }. Best is trial 202 with value: 0.698759913444519.


params = {}    
#     for i in range(len(final_preds)):
#         params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)

params[f"w1"] = 0.24573090810759818
#     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)

params[f"w2"] = 0.07968599547857583
#     params[f"w2"] = trial.suggest_float(f'w2', 0, 1)

params[f"w3"] = 0.3988612609534191
#     params[f"w3"] = trial.suggest_float(f'w3', 0, 1)

params[f"w4"] = 0.05807376299716156
#     params[f"w4"] = trial.suggest_float(f'w4', 0, 1)

params[f"w5"] = 0.8609716669901828
#     params[f"w5"] = trial.suggest_float(f'w5', 0, 1)
params[f"w6"] = 0.7386570274001879
#     params[f"w6"] = trial.suggest_float(f'w6', 0, 1)

params[f"w7"] = 0.9788696721148743
#     params[f"w7"] = trial.suggest_float(f'w7', 0, 1)
params[f"w8"] = 0.8270723521215965
#     params[f"w8"] = trial.suggest_float(f'w8', 0, 1)
params[f"w9"] = 0.5590153126204409
#     params[f"w9"] = trial.suggest_float(f'w9', 0, 1)
params[f"w10"] = 0.7265498790207406
#     params[f"w10"] = trial.suggest_float(f'w10', 0, 1)
params[f"w11"] = 0.41216752969852644
#     params[f"w11"] = trial.suggest_float(f'w11', 0, 1)
params[f"w12"] = 0.5864404449549478
#     params[f"w12"] = trial.suggest_float(f'w12', 0, 1)

params

In [ ]:
# {
#     'w1': 0.7458107541498612,
#     'w2': 0.2900793805943048,
#     'w3': 0.40430935291387626,
#     'w4': 0.12087058264990641,
#     'w5': 0.5101955071950133,
#     'w6': 0.28429946034177606,
#     'w7': 0.364869611492901,
#     'w8': 0.10441135348615381,
#     'w9': 0.48784730233361917,
#     'w10': 0.9105341528815705
# }

 # . Best is trial 289 with value: 0.6934514045715332.


# roh_01_tta roh_02_tta, roh_07_tta, roh_06, roh_07, 
# roh_03_tta, roh_06_tta, roh_03, roh_05, nir_06_tta, nir_01, nir_02
# roh_04_tta, roh_01, nir_02_tta, 131
# roh_02, nir_03_tta, nir_04_tta, nir_05_tta, nir_06, 160
# roh_04, 166
# nir_04, 160_tta
# nir_03, 127

In [ ]:
# params_pos = {
#  'w1': 0.655195343067219,
#  'w2': 0.2394705943310217,
#  'w3': 0.2927163844564814,
#  'w4': 0.4880033823349387,
#  'w5': 0.8387396842282611,
#  'w6': 0.9530480302035045,
#  'w7': 0.9573363671729265,
#  'w8': 0.6251416370174683,
#  'w9': 0.5371852737978187,
#  'w10': 0.4593912556686621,
#  'w11': 0.0,
#  'w12': 0.0
# }


# params_pos = {
#  'w1': 0.0,
#  'w2': 0.0,
#  'w3': 0.0,
#  'w4': 0.3074762360451622,
#  'w5': 0.0,
#  'w6': 0.12305169260913107,
#  'w7': 0.5363904743468609,
#  'w8': 0.0,
#  'w9': 0.8717967389387182,
#  'w10': 0.0,
#  'w11': 0.9798424243827817,
#  'w12': 0.7060773818241292
# }

params_pos = {
 'w1': 1.3387089493909258e-05,
 'w2': 0.7832907921668216,
 'w3': 0.1644769178230628,
 'w4': 0.00055845444967733,
 'w5': 0.2303089374666643,
 'w6': 0.05687648005878053,
 'w7': 0.5300038291077748,
 'w8': 0.24812342792556633,
 'w9': 0.06386465242721732,
 'w10': 0.2985146430301337,
 'w11': 0.9845085492621458,
 'w12': 0.5463258184950742
}



# params_all = {
#  'w1': 0.3154856556313033,
#  'w2': 0.2350959121391397,
#  'w3': 0.27103628342691727,
#  'w4': 0.6492473698434531,
#  'w5': 0.8397641633442563,
#  'w6': 0.8477025966292081,
#  'w7': 0.8096714084748394,
#  'w8': 0.4026009341206447,
#  'w9': 0.9969765714831549,
#  'w10': 0.39361450317658253,
#  'w11': 0.0,
#  'w12': 0.0
# }

params_all =  {
 'w1': 0.03644385570565463,
 'w2': 0.2254980335743082,
 'w3': 0.15865020296782992,
 'w4': 0.19357382597848447,
 'w5': 0.49930822369312966,
 'w6': 0.5592562127174933,
 'w7': 0.9150150266829656,
 'w8': 0.4294644367585147,
 'w9': 0.11137152838536817,
 'w10': 0.3808669569832525,
 'w11': 0.3506406085711101,
 'w12': 0.30010922179305505
}


model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).cuda() #.flatten().cuda() 


In [ ]:
len(final_preds)


def weighted_ensemble(params, final_preds):    
    for index, val in enumerate(params.keys()):
        if index == 0:
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    return preds

In [ ]:
# 0.70111567
for i in range(0, 300, 10): # 645
    preds_final = final_preds[0]

    for idx in range(final_preds[0].shape[0]):
        preds = [val[idx] for val in final_preds]
        a = weighted_ensemble(params_all, preds)
        if a.sum() > i:
            a = 0.9*a + 0.1*weighted_ensemble(params_pos, preds)
        preds_final[idx] = a

    preds_final = preds_final


    score = dice_coef(model_masks, preds_final, thr=0.5).cpu().detach().numpy() 
#     print(i, score)
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, preds_final, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    print(i, best_dice_score, best_threshold)

In [ ]:
len(final_preds)

In [ ]:
x = np.ones(final_preds[0].shape)

In [ ]:
for idx, val in enumerate(call_signs1):
    print(idx+1, val)

In [ ]:
params = {}    
#     for i in range(len(final_preds)):
#         params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)

#     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)
params[f"w1"] = 0.2559174030552294
#     params[f"w2"] = trial.suggest_float(f'w2', 0, 1)
params[f"w2"] = 0.17464567024483976
#     params[f"w3"] = trial.suggest_float(f'w3', 0, 1)
params[f"w3"] = 0.2901265133088786
#     params[f"w4"] = trial.suggest_float(f'w4', 0, 1)
params[f"w4"] = 0.5116453708319416
#     params[f"w5"] = trial.suggest_float(f'w5', 0, 1)
params[f"w5"] = 0.7560514736860692
#     params[f"w6"] = trial.suggest_float(f'w6', 0, 1)
params[f"w6"] = 0.2927534853839116
params

In [ ]:
# Trial 145 finished with value: 0.6954816579818726 and parameters: 
# {'w1': 0.3800712934060794, 'w2': 0.19827057881403873, 'w3': 0.8639190435791337, 'w4': 0.8183765509720466, 'w5': 0.17899313642679757, 'w6': 0.024590770644496746, 'w7': 0.7862484916619368, 'w8': 0.992834740398046, 'w9': 0.3995791428881191}. 
# Best is trial 145 with value: 0.6954816579818726. # 0.4

In [ ]:
# [I 2023-07-16 19:49:38,527] Trial 258 finished with value: 0.6955569982528687 and parameters: 
# {'w1': 0.35630544690551724, 'w2': 0.3882333167896021, 'w3': 0.793029020160463, 'w4': 0.589733873322586, 'w5': 0.1238533606039253, 'w6': 0.3136224869358406, 'w7': 0.7445672749651168, 'w8': 0.9585857088778065, 'w9': 0.3523881046014743}. Best is trial 258 with value: 0.6955569982528687.
# 0.41

In [ ]:
# [I 2023-07-16 20:01:35,463] Trial 231 finished with value: 0.6953996419906616 and parameters: {'w1': 0.03615687143100031, 'w2': 0.45224010503148204, 'w3': 0.663721473024527, 'w4': 0.6371224751418889, 'w5': 0.03698106916893698, 'w6': 0.15052361112796397, 'w7': 0.5701940304803765, 'w8': 0.910815487921195, 'w9': 0.47052561326641523}. Best is trial 231 with value: 0.6953996419906616.
# 0.42

In [ ]:
# CFGS1 = [   
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'model_func': load_model_exp28,
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#         'call_sign': "ioa_01",
#         'tta': True, #False, #True
#         'normalize': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "ioa_02",
#         'model_func': load_model_exp28_snapshot,
#         'tta': False, #True
#         'normalize': True,
#         'model_pth': [
#             ######## full train.csv 
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#             ######### 4-skf split 
#         ]
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "roh_02",
#         'model_func': load_model_exp28_snapshot,        
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#         'tta': False, #True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "roh_03",
#         'model_func': load_model_exp28_snapshot,        
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#         'tta': False, #True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "roh_04",
#         'model_func': load_model_exp28_snapshot,        
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#         'tta': False, #True
#     }, 
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "ioa_02",
#         'model_func': load_model_exp28_snapshot,
#         'tta': False, #True
#         'normalize': True,
#         'model_pth': [
#             ######## full train.csv 
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#             ######### 4-skf split 
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v1.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v2.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v3.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v4.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v5.ckpt'
#         ]
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "ioa_02_tta",
#         'model_func': load_model_exp28_snapshot,
#         'tta': True, #True
#         'normalize': True,
#         'model_pth': [
#             ######## full train.csv 
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#             ######### 4-skf split 
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v1.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v2.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v3.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v4.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v5.ckpt'
#         ]
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_tta",
#         'model_func': load_modelr1,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01",
#         'model_func': load_modelr1,
#         'tta': False
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'cs3darknet_l',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/cs3darknet_l-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_02_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'cs3edgenet_x',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/cs3edgenet_x-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_03_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'cs3sedarknet_l',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/cs3sedarknet_l-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_04_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'ecaresnet101d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/ecaresnet101d-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_05_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'ecaresnet269d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/ecaresnet269d-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_06_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'fbnetv3_g',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/fbnetv3_g-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_07_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
        
#     {
#         'model_name': 'Unet',
#         'backbone': 'tf_mixnet_l',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/tf_mixnet_l-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_08_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
    

#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01_tta",
#         'model_func': load_model4,
#         'tta': True
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02_tta",
#         'model_func': load_model2,
#         'tta': True
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_03_tta", 
#         'model_func': load_model3,
#         'tta': True
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04_tta", 
#         'model_func': load_model1,
#         'tta': True
#     }, 
    
    
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l2',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_eca_nfnet_l2_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_05_tta",
#         'model_func': load_model2,
#         'tta': True
#     },
#     {
#     'model_name': 'Unet++',
#         'backbone': 'densenetblur121d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_densenetblur121d_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_06_tta", 
#         'model_func': load_model2,
#         'tta': True
#     },
    
#     {
#     'model_name': 'Unet++',
#         'backbone': 'convnextv2_base',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/convnextfold0_convnextv2_base_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07_tta", 
#         'model_func': load_model2,
#         'tta': True
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01",
#         'model_func': load_model4,
#         'tta': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02",
#         'model_func': load_model2,
#         'tta': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_03", 
#         'model_func': load_model3,
#         'tta': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04", 
#         'model_func': load_model1,
#         'tta': False
#     }, 
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l2',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_eca_nfnet_l2_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_05",
#         'model_func': load_model2,
#         'tta': False
#     },
#     {
#     'model_name': 'Unet++',
#         'backbone': 'densenetblur121d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_densenetblur121d_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_06", 
#         'model_func': load_model2,
#         'tta': False
#     },
    
#     {
#     'model_name': 'Unet++',
#         'backbone': 'convnextv2_base',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/convnextfold0_convnextv2_base_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07", 
#         'model_func': load_model2,
#         'tta': False
#     },
    
    
    
    
    
# ]

In [ ]:
cfg = [       
    "roh_02_tta", 
] 


# model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).flatten().cuda() 
model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).flatten().cuda() 


preds = []

for idx, sign in tqdm(enumerate(cfg), total=len(cfg)):
    wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy"))
    
    
#     wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).flatten().cuda()  
    
    

In [ ]:
wt.shape

In [ ]:
for idx, val in enumerate(wt):
#     print(val.sum() <= 10)
    if val.sum() <= 10:
        wt[idx, :, :] = 0
    
# print("mhsdbc")

In [ ]:
# np.sum(val)

In [ ]:
wt = wt.flatten().cuda()
score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
print("0.5 TH Score: ", score)

best_threshold = 0.0
best_dice_score = 0.0
for threshold in [i / 100 for i in range(101)] :
    score = dice_coef(model_masks, wt, thr=threshold).cpu().detach().numpy() 
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold
print(best_dice_score, best_threshold)

In [ ]:
# 0.67922837

In [ ]:
def boost_dice(params, TH=None):    
    
    for index, val in enumerate(params.keys()):
        if index == 0:            
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
    
#     for i, val in enumerate(preds):
#         if val.sum() <= 26:
#             preds[i, :, :] = 0
    
#     print(preds.shape, model_masks.shape)
#     get_small_area(preds, model_masks)
#     print(preds.shape)    
    
    
    if TH:
        best_threshold = TH
        preds1 = (preds).double()
        best_score = dice_coef(model_masks, preds1, thr=TH).cpu().detach().numpy() # get_score(labels, preds) 
    else:
        best_threshold = 0.0
        best_score = 0.0
        for threshold in [i / 100 for i in range(101)] :              
            if threshold < 0.3 or threshold > 0.65:
                continue

            preds1 = ((preds)>threshold).double()            
            score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
            if score > best_score:
                best_score = score
                best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [ ]:
call_signs1 = [    
    "roh_10_tta",
#     "roh_02_tta", 
    "nir_05",
    "nir_06_tta",
    "nir_09",
    "121_tta",
    "162_tta",
    "166_tta",
    "125", 
    "196",
    "127",
    
    "204",
    "roh_11_tta_full",
    "roh_12_tta"
    
    # above this 
    
#     "195",
#     "198",
#     "184", # 1
#     "174",
#     "150",
#     "138"    
    
        
#     "195_tta",
#     "198_tta",
#     "184_tta",
#     "174_tta",
#     "150_tta",
#     "138_tta"
    
    
] 

# 

# call_signs1 = [        
#     "roh_10_tta",
# #     "roh_02_tta", 
#     "roh_11_tta_full",

    
#     "nir_05",
#     "nir_06_tta",
#     "nir_09",
#     "121_tta",
#     "162_tta",
#     "166_tta",
#     "125", 
#     "196",
#     "127",
    
#     "204",
    
# ]

call_signs1 = [    
#     "roh_10_tta",
#     "roh_02_tta", 
    "roh_12_tta",
    "roh_02_tta",
    "roh_11_tta",
    "nir_05_tta",
    "nir_06_tta",
    "nir_09_tta",
    "121_tta",
    "162_tta",
    "166_tta",
    "125_tta", 
    "196_tta",
#     "127_tta",
    "204", # tta
    
    "Unetpp-convnext_base_preds_tta_06638",
    "230_tta"
    
#     "Unet-tf_efficientnet_l2_ns_tta_06570"
    # above this 
    
#     "nir_09"
    
#     "195_tta",
#     "198_tta",
#     "184_tta",
#     "174_tta",
#     "150_tta",
#     "138_tta"
] 



val_df = pd.read_csv("../../input/data_utils/val_df_filled.csv")

ix = val_df.loc[val_df['class'] == 1].index.values

model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).cuda() #.flatten().cuda() 
model_masks = model_masks[ix]
preds = []

sharp = 0.55
areas = [31, 15, 22, 31, 8, 10, 42, 13]

for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
#     if idx == 0:
#         wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.npy")).cuda()
#     else:
    wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).cuda() 

    wt = wt[ix]
    preds.append(wt)    
    score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
    print(score)
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, wt, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    print(best_dice_score, best_threshold)
    
final_preds = preds


In [ ]:
def objective(trial):
    TH = None #0.38 # None #0.39 # None #0.39
    
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)

    score, best_threshold  = boost_dice(params, TH=TH)
    params['threshold'] = best_threshold
    if not TH:
        print(params)
        
    return score

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)
best_params = study.best_params

In [ ]:
best_params # 0.36 | 0.710804283618927

In [ ]:
call_signs1 = [
    "/home/rohits/pv1/Contrail_Detection/output/pseudo_preds/roh_10_tta_3.pt",
    "/home/rohits/pv1/Contrail_Detection/output/pseudo_preds/roh_10_tta_4.npy",
    "/home/rohits/pv1/Contrail_Detection/output/pseudo_preds/roh_10_tta_5.pt"
]

model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).cuda() #.flatten().cuda() 


preds = []
for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
    if idx != 1:
        wt = torch.load(sign).cuda()
    else:
        wt = torch.from_numpy(np.load(sign)).cuda()
        
    preds.append(wt)

In [ ]:
model_masks.shape, preds[0].shape

In [ ]:
for pred in preds:
    score = dice_coef(model_masks, pred, thr=0.5).cpu().detach().numpy() 
    print(score)
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, pred, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    print(best_dice_score, best_threshold)

In [ ]:
x1 = (preds[0] > 0.19).double()
x2 = (preds[1] > 0.19).double()
x3 = (preds[2] > 0.19).double()

In [ ]:
sum1 = []
for a in x1:
    sum1.append(a.sum().cpu().numpy().item())
    
    
sum2 = []
for a in x2:
    sum2.append(a.sum().cpu().numpy().item())
    
    
    
sum3 = []
for a in x3:
    sum3.append(a.sum().cpu().numpy().item())
    
    
sum4 = []
for a in model_masks:
    sum4.append(a.sum().cpu().numpy().item())

In [ ]:
fps = []
for idx, (a, b, c, d) in enumerate(zip(sum1, sum2, sum3, sum4)):
    print(a, b, c, d)
    if idx > 1:
        break
    
    if a == b == c == 0 and d != 0:
        fps.append(d)
    

In [ ]:
len(fps)

In [ ]:
x1 = x1 #.flatten()
x2 = x2 #.flatten()
x3 = x3 #.flatten()

In [ ]:
x = (x1 + x2 + x3)
    
    
x = (x >= 1) == 1

In [ ]:
score = dice_coef(model_masks, x, thr=0.5).cpu().detach().numpy() 
score

In [ ]:
x1[2], x2[2], x3[2]

In [ ]:
# model_masks

import numpy as np

# def merge_predictions(mask_t_minus_1, mask_t, mask_t_plus_1, time_shift):
#     # Ensure all three masks have the same dimensions (256x256)
#     assert mask_t_minus_1.shape == mask_t.shape == mask_t_plus_1.shape == (256, 256), "All masks must have size 256x256"

#     # Create an empty merged mask with zeros
#     merged_mask = np.zeros((256, 256))

#     # Time shift should be within the bounds of the mask dimensions
#     time_shift = max(-1, min(1, time_shift))

#     # Calculate the shift directions for t-1, t, and t+1 masks
#     shift_t_minus_1 = (0, 0) if time_shift == 1 else (time_shift, time_shift)
#     shift_t_plus_1 = (0, 0) if time_shift == -1 else (-time_shift, -time_shift)

#     # Merge the predictions while considering the time shift
#     merged_mask = np.logical_or(merged_mask, np.roll(mask_t_minus_1, shift_t_minus_1, axis=(0, 1)))
#     merged_mask = np.logical_or(merged_mask, mask_t)
#     merged_mask = np.logical_or(merged_mask, np.roll(mask_t_plus_1, shift_t_plus_1, axis=(0, 1)))

#     return merged_mask

# Example usage:
# Assume mask_t_minus_1, mask_t, mask_t_plus_1 are numpy arrays representing the masks
# time_shift is an integer representing the time shift (-1, 0, or 1)
# merged_mask = merge_predictions(mask_t_minus_1, mask_t, mask_t_plus_1, time_shift)


def merge_predictions(mask_t_minus_1, mask_t, mask_t_plus_1, time_shift):
    # Ensure all three masks have the same dimensions (1856, 256, 256)
    assert mask_t_minus_1.shape == mask_t.shape == mask_t_plus_1.shape == (1856, 256, 256), "All masks must have shape (1856, 256, 256)"

    # Create an empty merged mask with zeros
    merged_mask = np.zeros((1856, 256, 256))

    # Time shift should be within the bounds of the mask dimensions
    time_shift = max(-1, min(1, time_shift))

    # Calculate the shift directions for t-1, t, and t+1 masks
    shift_t_minus_1 = (0, 0) if time_shift == 1 else (time_shift, time_shift)
    shift_t_plus_1 = (0, 0) if time_shift == -1 else (-time_shift, -time_shift)

    # Merge the predictions while considering the time shift for each frame
    for i in range(1856):
        merged_mask[i] = np.logical_or(merged_mask[i], np.roll(mask_t_minus_1[i], shift_t_minus_1, axis=(0, 1)))
        merged_mask[i] = np.logical_or(merged_mask[i], mask_t[i])
        merged_mask[i] = np.logical_or(merged_mask[i], np.roll(mask_t_plus_1[i], shift_t_plus_1, axis=(0, 1)))

    return merged_mask




In [ ]:
for time_shift in range(256):
    x = torch.from_numpy(merge_predictions(x1.cpu().numpy(),x2.cpu().numpy(),x3.cpu().numpy(), time_shift)).cuda()

    score = dice_coef(model_masks[idx], x, thr=0.5).cpu().detach().numpy() 
    print(score)
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, x, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    print(best_dice_score, best_threshold)
        

In [ ]:
model_masks[0]

In [ ]:
x1.shape